<a href="https://colab.research.google.com/github/Cami050/proyecto_exploracion_datos/blob/main/Proyecto_de_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Instalar gdown (un gestor de paquetes de Python)si aún no está instalado
!pip install gdown -q

In [ ]:
import gdown
import pandas as pd

# ID del archivo obtenido del enlace público
file_id = '1bdk3h8xRW-hcZ1P1s_R9h64neuOAa-6y'
output_file = 'DELITOS_INFORMÁTICOS_20251209.csv'

# URL de descarga directa de Google Drive
url = f'https://drive.google.com/uc?id={file_id}'

# Descargar el archivo
gdown.download(url, output_file, quiet=False)

# Cargar el archivo CSV en un DataFrame de pandas
df = pd.read_csv(output_file)

# Mostrar las primeras 5 filas del DataFrame
display(df.head())

Downloading...
From: https://drive.google.com/uc?id=1bdk3h8xRW-hcZ1P1s_R9h64neuOAa-6y
To: /content/DELITOS_INFORMÁTICOS_20251209.csv
100%|██████████| 56.2M/56.2M [00:00<00:00, 65.3MB/s]


,FECHA HECHO,COD_DEPTO,DEPARTAMENTO,COD_MUNI,MUNICIPIO,DESCRIPCION CONDUCTA,CANTIDAD
0,2006 May 13 12:00:00 AM,11,BOGOTA D.C.,11001,BOGOTA D.C.,ARTICULO 269I. HURTO POR MEDIOS INFORMATICOS Y...,1
1,2006 Oct 06 12:00:00 AM,11,BOGOTA D.C.,11001,BOGOTA D.C.,ARTICULO 269A. ACCESO ABUSIVO A UN SISTEMA INF...,1
2,2008 May 24 12:00:00 AM,11,BOGOTA D.C.,11001,BOGOTA D.C.,ARTICULO 269A. ACCESO ABUSIVO A UN SISTEMA INF...,1
3,2009 Jan 01 12:00:00 AM,41,HUILA,41001,NEIVA,ARTICULO 269F. VIOLACION DE DATOS PERSONALES,1
4,2009 Jan 02 12:00:00 AM,11,BOGOTA D.C.,11001,BOGOTA D.C.,ARTICULO 269A. ACCESO ABUSIVO A UN SISTEMA INF...,1


In [7]:
"""
PROYECTO FINAL - DELITOS INFORMÁTICOS COLOMBIA
==============================================
FASE 2: PROCESAMIENTO DE FECHAS
Responsable: CAMILA RIVERA
Pregunta: ¿Cómo ha evolucionado el número total de delitos por año?
"""

print("="*70)
print("FASE 2: PROCESAMIENTO DE DATOS TEMPORALES")
print("RESPONSABLE: CAMILA RIVERA")
print("Pregunta guía: Evolución temporal de delitos")
print("="*70)

# 1. Identificar columna de fecha
print("PASO 1: IDENTIFICANDO COLUMNA DE FECHA...")

# Buscar columna que contenga 'fecha' en el nombre
columnas_fecha = [col for col in df.columns if 'fecha' in col.lower()]
if columnas_fecha:
    fecha_col = columnas_fecha[0]
    print(f"Columna identificada: '{fecha_col}'")
else:
    print(" No se encontró columna de fecha")
    # Mostrar todas las columnas para identificar manualmente
    print("Columnas disponibles:")
    for i, col in enumerate(df.columns, 1):
        print(f"{i:2d}. {col}")
    fecha_col = input("\n ¿Cuál es la columna de fecha? (escribe el nombre): ")

# 2. Convertir a formato datetime
print(f"\n PASO 2: CONVIRTIENDO '{fecha_col}' A DATETIME...")

# Guardar copia del estado original
fecha_original = df[fecha_col].copy()

# Intentar diferentes formatos de fecha
try:
    df[fecha_col] = pd.to_datetime(df[fecha_col], errors='coerce')
    print("Conversión exitosa")
except Exception as e:
    print(f"Error: {e}")
    print("Intentando con formato específico...")
    try:
        df[fecha_col] = pd.to_datetime(df[fecha_col], format='%Y-%m-%d', errors='coerce')
        print("Conversión con formato YYYY-MM-DD exitosa")
    except:
        print("No se pudo convertir la fecha")

# 3. Verificar conversión
conversiones_exitosas = df[fecha_col].notna().sum()
porcentaje_exito = (conversiones_exitosas / len(df)) * 100
print(f"\n ESTADÍSTICAS DE CONVERSIÓN:")
print(f"• Filas convertidas: {conversiones_exitosas:,} de {len(df):,}")
print(f"• Porcentaje éxito: {porcentaje_exito:.1f}%")
print(f"• Rango temporal: {df[fecha_col].min()} a {df[fecha_col].max()}")

# 4. Crear columnas derivadas (AÑO, MES, MES_AÑO)
print("\n PASO 3: CREANDO COLUMNAS DERIVADAS...")

df['AÑO'] = df[fecha_col].dt.year
df['MES'] = df[fecha_col].dt.month
df['DIA'] = df[fecha_col].dt.day
df['MES_AÑO'] = df[fecha_col].dt.strftime('%Y-%m')
df['DIA_SEMANA'] = df[fecha_col].dt.day_name()

print("Columnas creadas:")
print(f"  • AÑO: {df['AÑO'].nunique()} años únicos")
print(f"  • MES: {df['MES'].nunique()} meses únicos")
print(f"  • MES_AÑO: {df['MES_AÑO'].nunique()} períodos únicos")

# 5. Análisis de distribución temporal
print("\n PASO 4: ANÁLISIS DE DISTRIBUCIÓN TEMPORAL")

# Distribución por año
delitos_por_año = df['AÑO'].value_counts().sort_index()
print("\n DISTRIBUCIÓN POR AÑO:")
for año, cantidad in delitos_por_año.items():
    if not pd.isna(año):
        print(f"  • {int(año)}: {cantidad:,} casos")

# Distribución por mes
print("\n DISTRIBUCIÓN POR MES (promedio anual):")
meses_es = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
            'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

for mes_num in range(1, 13):
    cantidad = df[df['MES'] == mes_num].shape[0]
    promedio = cantidad / df['AÑO'].nunique() if df['AÑO'].nunique() > 0 else 0
    if cantidad > 0:
        print(f"  • {meses_es[mes_num-1]}: {cantidad:,} casos ({promedio:.0f} por año)")

print("\n" + "="*70)
print("FASE 2 COMPLETADA - CAMILA RIVERA")
print("Fechas procesadas y columnas derivadas creadas")
print("Próximo paso: Andrés Padilla - Limpieza geográfica")
print("="*70)

FASE 2: PROCESAMIENTO DE DATOS TEMPORALES
RESPONSABLE: CAMILA RIVERA
Pregunta guía: Evolución temporal de delitos
PASO 1: IDENTIFICANDO COLUMNA DE FECHA...
Columna identificada: 'FECHA HECHO'

 PASO 2: CONVIRTIENDO 'FECHA HECHO' A DATETIME...


/tmp/ipython-input-1889715728.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[fecha_col] = pd.to_datetime(df[fecha_col], errors='coerce')


Conversión exitosa

 ESTADÍSTICAS DE CONVERSIÓN:
• Filas convertidas: 446,326 de 446,326
• Porcentaje éxito: 100.0%
• Rango temporal: 2006-05-13 00:00:00 a 2025-10-31 00:00:00

 PASO 3: CREANDO COLUMNAS DERIVADAS...
Columnas creadas:
  • AÑO: 19 años únicos
  • MES: 12 meses únicos
  • MES_AÑO: 205 períodos únicos

 PASO 4: ANÁLISIS DE DISTRIBUCIÓN TEMPORAL

 DISTRIBUCIÓN POR AÑO:
  • 2006: 2 casos
  • 2008: 1 casos
  • 2009: 471 casos
  • 2010: 844 casos
  • 2011: 1,564 casos
  • 2012: 3,180 casos
  • 2013: 4,629 casos
  • 2014: 3,676 casos
  • 2015: 7,404 casos
  • 2016: 8,651 casos
  • 2017: 15,057 casos
  • 2018: 21,279 casos
  • 2019: 22,092 casos
  • 2020: 49,359 casos
  • 2021: 52,224 casos
  • 2022: 61,993 casos
  • 2023: 63,249 casos
  • 2024: 72,996 casos
  • 2025: 57,655 casos

 DISTRIBUCIÓN POR MES (promedio anual):
  • Enero: 34,629 casos (1823 por año)
  • Febrero: 35,040 casos (1844 por año)
  • Marzo: 36,179 casos (1904 por año)
  • Abril: 36,947 casos (1945 por año)
  